In [ ]:
import os
from scipy.io import loadmat
import pandas as pd
import kagglehub


## General rules
1. update group when working and what sections
2. Try to be comprehensive as you write! Leave short markdown descriptions for the average reader to understand what you're doing

## Overview
### [FILL IN WITH PROJECT DESCRIPTION]

## Data

In [ ]:
# Download latest version
path = kagglehub.dataset_download("eduardo4jesus/stanford-cars-dataset")

print("Path to dataset files:", path)

100%|██████████| 1.82G/1.82G [05:29<00:00, 5.94MB/s]

Extracting files...


Path to dataset files: /Users/blackbirdsix/.cache/kagglehub/datasets/eduardo4jesus/stanford-cars-dataset/versions/1


In [14]:
print("Path:", path)
print("Folders/files inside the dataset:")
print(os.listdir(path))
devpath = path + '/car_devkit/devkit'
print(os.listdir(devpath))
metapath = devpath + "/cars_meta.mat"

Path: /Users/blackbirdsix/.cache/kagglehub/datasets/eduardo4jesus/stanford-cars-dataset/versions/1
Folders/files inside the dataset:
['cars_train', 'cars_test', 'car_devkit']
['cars_test_annos.mat', 'eval_train.m', 'cars_meta.mat', 'README.txt', 'cars_train_annos.mat', 'train_perfect_preds.txt']


In [ ]:
meta = loadmat(metapath)
class_names = [c[0] for c in meta["class_names"][0]]

# Put into a DataFrame
df = pd.DataFrame(class_names, columns=["class_name"])

print(df['class_name'])


0            AM General Hummer SUV 2000
1                   Acura RL Sedan 2012
2                   Acura TL Sedan 2012
3                  Acura TL Type-S 2008
4                  Acura TSX Sedan 2012
                     ...               
191    Volkswagen Beetle Hatchback 2012
192            Volvo C30 Hatchback 2012
193                Volvo 240 Sedan 1993
194                 Volvo XC90 SUV 2007
195       smart fortwo Convertible 2012
Name: class_name, Length: 196, dtype: object


## Supervised Learning
### [ QUICK OVERVIEW OF SECTION ]

## Unsupervised Learning
### [QUICK OVERVIEW OF SECTION]